In [12]:
import pandas as pd

In [13]:
#trazendo os dados da bronze
path_customer_bronze = "C:\\Users\\jhoe.hashimoto\\Desktop\\dados_teste\\bronze\\customer_data.parquet"
path_sales_bronze = "C:\\Users\\jhoe.hashimoto\\Desktop\\dados_teste\\bronze\\sales_data.parquet"

*ABERTURA E LEITURA DE ARQUIVO*

In [14]:
#lendo os datasets
df_customer = pd.read_parquet(path_customer_bronze)
df_sales = pd.read_parquet(path_sales_bronze)

*INÍCIO ANÁLISE EXPLORATÓRIA*

In [15]:
#verificando dados da customer
df_customer.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  100 non-null    int64 
 1   nome        100 non-null    object
 2   sobrenome   100 non-null    object
 3   email       100 non-null    object
 4   telefone    100 non-null    object
 5   endereço    100 non-null    object
 6   numero      100 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 5.6+ KB


In [16]:
#verificando dados da sales
df_sales.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      100 non-null    int64         
 1   product_name  100 non-null    object        
 2   quantity      100 non-null    int64         
 3   unit_price    100 non-null    int64         
 4   cliente_id    100 non-null    int64         
 5   order_date    100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 4.8+ KB


In [ ]:
# duplicados = df_sales.duplicated(subset=['order_id'])
# # Exibindo as linhas duplicadas (marcadas como True)
# duplicados

In [ ]:
# #verificando duplicidade no dataframe, através da chave order_id
# contagem_por_valor = df_sales['order_id'].value_counts()
# contagem_por_valor

In [ ]:
# #verificando duplicidade de chave no dataframe, através da chave id_cliente
# contagem_por_valor = df_customer['id_cliente'].value_counts()
# contagem_por_valor

In [106]:
# #apagando células vazias 
# df_sales.dropna(inplace = True)
# df_customer.dropna(inplace = True)  ####

In [ ]:
#rename df_customer
df_consumidores=df_customer.rename(columns={'id_cliente':'id_cliente',
                                            'nome': 'nome', 'sobrenome': 'sobrenome', 'email':'email', 'telefone': 'telefone',
                                            'endereço' : 'endereco', 'numero' :'numero'})
df_consumidores.head()

In [ ]:
#rename df_sales
df_vendas=df_sales.rename(columns={'order_id' : 'order_id', 
                                   'product_name':'product_name', 'quantity':'quantity', 'unit_price':'unit_price', 
                                   'cliente_id': 'id_cliente'  ,'order_date':'order_date'})
df_vendas.head()

*Teste Salvando em CSV*

In [111]:
# df_vendas.to_csv('vendas.csv', index=False)
# df_consumidores.to_csv('consumidores.csv', index=False)

In [ ]:
################ MERGE PANDAS =! (INSERT, UPDATE, DELETE) ###############################
# df_combined = pd.merge(df_consumidores, df_vendas, on='id_cliente')
# df_combined.head()

*CONEXÃO COM O MYSQL*

In [ ]:
!pip install mysql-connector-python

In [21]:
from mysql.connector import connect
import mysql.connector
from mysql.connector import errorcode

In [28]:
#dados servidor
config = {
    'user': 'root',
    'password': 'jhoehashimoto',
    'host': 'localhost',
    'database': 'v2silver_sales_customer'
}

*CRIANDO AS TABELAS*

In [29]:
#Tabela Customer

try:
    # Criando a conexão
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()

        # Criando a tabela
        customer_silver = """
                        CREATE TABLE customer_silver (
                            id_cliente int NOT NULL,
                            nome varchar(50),
                            sobrenome varchar(50),
                            email varchar(50),
                            telefone varchar(20),
                            endereco varchar(50),
                            numero int,
                            PRIMARY KEY (id_cliente)
                        ) 
        """
        
        cursor.execute(customer_silver)
        print('Tabela criada com sucesso')
        
        # Fechando o cursor e a conexão
        cursor.close()
        connection.close()
    else:
        print('Não foi possível conectar ao banco de dados')
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("A tabela já existe.")
    else:
        print(f"Erro: {err.msg}")
finally:
    # Garantir que a conexão seja fechada se aberta
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

Tabela criada com sucesso


In [30]:
#Tabela SALES 
try:
    # Criando a conexão
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()

        # Criando a tabela
        sales_silver = """
                        CREATE TABLE sales_silver (
                        order_id int NOT NULL,
                        product_name varchar(50),
                        quantity int,
                        unit_price int,
                        id_cliente int,
                        order_date date,
                        PRIMARY KEY (order_id),
                        FOREIGN KEY (id_cliente) REFERENCES customer_silver (id_cliente)
                    ) 
        """
        cursor.execute(sales_silver)
        print('Tabela criada com sucesso')
        
        # Fechando o cursor e a conexão
        cursor.close()
        connection.close()
    else:
        print('Não foi possível conectar ao banco de dados')
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("A tabela já existe.")
    else:
        print(f"Erro: {err.msg}")
finally:
    # Garantir que a conexão seja fechada se aberta
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

Tabela criada com sucesso


*Conversão Tipagem do pandas para o mysql*

In [31]:
import numpy as np
def pandas_to_mysql_type(value):
    if isinstance(value, np.int64):
        return int(value)
    elif isinstance(value, np.float64):
        return float(value)
    elif isinstance(value, pd.Timestamp):
        return value.to_pydatetime().strftime('%Y-%m-%d %H:%M:%S')
    elif isinstance(value, pd.Series):
        return value.values.tolist()
    else:
        return value

In [32]:
#INSERT tabela customer_silver

try:
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()
        
        # Inserir os dados da tabela df_consumidores na tabela customer_silver
        for i in df_consumidores.index:
            query = """
            INSERT INTO customer_silver (id_cliente, nome, sobrenome, email, telefone, endereco, numero)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            data = (
                pandas_to_mysql_type(df_consumidores['id_cliente'][i]),
                df_consumidores['nome'][i],
                df_consumidores['sobrenome'][i],
                df_consumidores['email'][i],
                df_consumidores['telefone'][i],
                df_consumidores['endereco'][i],
                pandas_to_mysql_type(df_consumidores['numero'][i])
            )
            cursor.execute(query, data)
        
        # Commit das alterações e fechamento do cursor
        connection.commit()
        cursor.close()
        print('Dados inseridos com sucesso')
    
except mysql.connector.Error as err:
    print(f"Erro: {err}")
    if connection.is_connected():
        connection.rollback()
finally:
    # Fechamento da conexão
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

Dados inseridos com sucesso
Conexão ao MySQL fechada


In [ ]:
#INSERT tabela sales_silver

try:
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()
        
        # Inserir os dados da tabela df_vendas na tabela sales_silver
        for i in df_vendas.index:
            query = """
            INSERT INTO sales_silver (order_id, product_name, quantity, unit_price, id_cliente, order_date)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            data = (
                pandas_to_mysql_type(df_vendas['order_id'][i]),
                df_vendas['product_name'][i],
                pandas_to_mysql_type(df_vendas['quantity'][i]),
                pandas_to_mysql_type(df_vendas['unit_price'][i]),
                pandas_to_mysql_type(df_vendas['id_cliente'][i]),
                pandas_to_mysql_type(df_vendas['order_date'][i]),
            )
            
            # Verifica se o order_id já existe na tabela
            check_query = "SELECT order_id FROM sales_silver WHERE order_id = %s"
            cursor.execute(check_query, (data[0],))
            result = cursor.fetchone()
            
            if not result:
                cursor.execute(query, data)
                print(f'Dados inseridos com sucesso para order_id {data[0]}')
            else:
                print(f'Registro com order_id {data[0]} já existe, pulando inserção.')
        
        # Commit das alterações e fechamento do cursor
        connection.commit()
        cursor.close()
        print('Commit realizado com sucesso')
    
except mysql.connector.Error as err:
    print(f"Erro MySQL: {err}")
    if connection.is_connected():
        connection.rollback()
finally:
    # Fechamento da conexão
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')